In [51]:
# import statements
import os
import sys
from liquidhandling import SoloSoft, SoftLinx
from liquidhandling import *  # replace with the plate types you used 
from liquidhandling import DeepBlock_96VWR_75870_792_sterile
import pandas


In [53]:

# VARIABLES YOU CAN CHANGE ---------------------------------------------------------------------------------------------------------------

source_plate_files = [ # format = (unique plate name, csv file full path) 
    ("plate1", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_1.csv"), 
    ("plate2", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_2.csv"), 
    ("plate3", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_3.csv"),
    ("plate4", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_4.csv"), 
    ("plate5", "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/test_plate_files/plate_csv_5.csv")
]

save_to_directory_path = "/Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files"   # path to the folder where you want to save all created files
softLinx_basename = "cherry_picking_test"  # don't include the .slvp extension
asp_source_volume = 20
source_plates_per_round = 4

# ------------------------------------------------------------------------------------------------------------------------------------------




# calculate the number of rounds needed
num_rounds = int(len(source_plate_files)/source_plates_per_round)
num_rounds = num_rounds + 1 if (len(source_plate_files)%source_plates_per_round) else num_rounds


possible_source_dec_locs = [3,4,5,6] # do not change these!
destination_dictionary = {
    1:'A1', 2:'A2', 3:'A3', 4:'A4', 5:'A5', 6:'A6', 7:'A7', 8:'A8', 9:'A9', 10:'A10', 11:'A11', 12:'A12', 
    13:'B1', 14:'B2', 15:'B3', 16:'B4', 17:'B5', 18:'B6', 19:'B7', 20:'B8', 21:'B9', 22:'B10', 23:'B11', 24:'B12', 
    25:'C1', 26:'C2', 27:'C3', 28:'C4', 29:'C5', 30:'C6', 31:'C7', 32:'C8', 33:'C9', 34:'C10', 35:'C11', 36:'C12',
    37:'D1', 38:'D2', 39:'D3', 40:'D4', 41:'D5', 42:'D6', 43:'D7', 44:'D8', 45:'D9', 46:'D10', 47:'D11', 48:'D12',
    49:'E1', 50:'E2', 51:'E3', 52:'E4', 53:'E5', 54:'E6', 55:'E7', 56:'E8', 57:'E9', 58:'E10', 59:'E11', 60:'E12',
    61:'F1', 62:'F2', 63:'F3', 64:'F4', 65:'F5', 66:'F6', 67:'F7', 68:'F8', 69:'F9', 70:'F10', 71:'F11', 72:'F12',
    73:'G1', 74:'G2', 75:'G3', 76:'G4', 77:'G5', 78:'G6', 79:'G7', 80:'G8', 81:'G9', 82:'G10', 83:'G11', 84:'G12',
    85:'H1', 86:'H2', 87:'H3', 88:'H4', 89:'H5', 90:'H6', 91:'H7', 92:'H8', 93:'H9', 94:'H10', 95:'H11', 96:'H12',
} 

# extract plate_id and source wells from source plate csvs
transfers = []
for source_plate in source_plate_files: 
    plate_id, path = source_plate
        
    # read contents of each csv
    with open(path, "r", encoding='utf-8-sig') as open_csv:  # using 'utf-8-sig' ignores the \ufeff (what is that?)
        contents = open_csv.readlines()
        contents = [x.strip() for x in contents] 
        transfers.append((plate_id, contents))
        
# keep track of destination well and destination plate_location

transfers_list_copy = transfers.copy()  # copy of source plate array that I can mess with
dispense_count = 1
num_dipsense_plates = 1
is_dispense_plate_7 = True # True if dispensing into plate at position 7, false if dispensing into plate at position 8 (will flip as new plates are needeed)



for i in range(num_rounds): 
    # initialize SoftLinx (one softLinx file per round)
    softLinx_round_filename = f"{softLinx_basename}_ROUND{i+1}.slvp"
    softLinx_round_path = os.path.join(save_to_directory_path, softLinx_round_filename)
    softLinx = SoftLinx(softLinx_round_filename, softLinx_round_path)
    
    print(f"Round {i+1}: (output file = {softLinx_round_path} ")
    if len(transfers_list_copy) >= source_plates_per_round: 
        transfers_in_round = transfers_list_copy[:source_plates_per_round]
        transfers_list_copy = transfers_list_copy[source_plates_per_round:]
    else: 
        transfers_in_round = transfers_list_copy
        transfers_list_copy = []
        
    # Program variables
    tip_type = "TipBox.50uL.Axygen-EV-50-R-S.tealbox"
    source_plate_type = "DeepBlock.96.VWR-75870-792.sterile"
    destination_plate_type = "DeepBlock.96.VWR-75870-792.sterile"
    transfer_volume = 20
    default_z_shift = 2
#     blowoff = 5
        
    # loop through each set of transfers 
    for j in range(len(transfers_in_round)):
        plate_id, wells = transfers_in_round[j]
        plate_deck_location = possible_source_dec_locs[j]
        print(f"\tDeck location {plate_deck_location} --> {plate_id} = {wells} ")
        
        soloSoft_file_sep = 0
        
        for k in range(len(wells)):  
            # make a separate SoloSoft file per 15 transfers
            if k%15 == 0: 
                soloSoft_file_sep += 1
                current_filename = os.path.join(save_to_directory_path, f"round{i+1}_{plate_id}_step{soloSoft_file_sep}.hso")  # set filename for this round's solosoft file 
                print(f"\t\toutput filename = {current_filename})")
            
                 # initialize SoloSoft deck and filenames
                soloSoft = SoloSoft(
                    filename=current_filename,
                    plateList=[
                        tip_type,
                        "Empty",
                        source_plate_type,
                        source_plate_type,
                        source_plate_type,
                        source_plate_type,
                        destination_plate_type,
                        destination_plate_type,
                    ],
                )
                
            wells[k] = wells[k].strip() # remove any extra white spaces 
            soloSoft.getTip(num_tips=1)  # get only one tip
            
            # aspirate from specified well in specified source plate
            #TODO: Deep well version of the plates won't work so I had to use the reservoir version... should be fine
            soloSoft.aspirate(
                position=f"Position{plate_deck_location}",
                aspirate_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setCell(wells[k][0], int(wells[k][1:]), transfer_volume),
                aspirate_shift=[0, 0, default_z_shift],
            )

            # determine dispensing plate and well locations
            dispense_well = destination_dictionary[dispense_count]
            dispense_count += 1
            if dispense_count == 97: 
                dipsense_count = 1
                is_dispense_plate_7 = False if is_dispense_plate_7 else True
                num_dispense_plates += 1
            dipsense_plate_pos = "Position7" if is_dispense_plate_7 else "Position8"
            print(f"\t\tasp: {wells[k]} --> disp: {dispense_well}")
            
            # dispense into determined plate/well 
            soloSoft.dispense(
                position=dipsense_plate_pos,
                dispense_volumes=Reservoir_12col_Agilent_201256_100_BATSgroup().setCell(dispense_well[0], int(dispense_well[1:]), transfer_volume), 
                dispense_shift=[0,0,default_z_shift],
            )
            
            # shuck tip twice becuase sometimes the small tip gets stuck 
            soloSoft.shuckTip()
            soloSoft.shuckTip()
            soloSoft.savePipeline()
        softLinx.soloSoftRun(current_filename)

    softLinx.saveProtocol()      
            
print("\nPROTOCOL INFORMATION: --------------------------------------")         
print(f"\tNumber of source plates: {len(source_plate_files)}")
print(f"\tNumber of destination plates = {num_dipsense_plates}")  
print(f"\tNumber of rounds: {num_rounds}") 


Round 1: (output file = /Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files/cherry_picking_test_ROUND1.slvp 
	Deck location 3 --> plate1 = ['A1', 'A3', 'A6', 'B2', 'C5', 'C6', 'C7', 'D1', 'E3', 'E4', 'E7', 'E9', 'E10', 'E11', 'E12', 'F2', 'G1', 'H1', 'H12'] 
		output filename = /Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files/round1_plate1_step1.hso)
		asp: A1 --> disp: A1
		asp: A3 --> disp: A2
		asp: A6 --> disp: A3
		asp: B2 --> disp: A4
		asp: C5 --> disp: A5
		asp: C6 --> disp: A6
		asp: C7 --> disp: A7
		asp: D1 --> disp: A8
		asp: E3 --> disp: A9
		asp: E4 --> disp: A10
		asp: E7 --> disp: A11
		asp: E9 --> disp: A12
		asp: E10 --> disp: B1
		asp: E11 --> disp: B2
		asp: E12 --> disp: B3
		output filename = /Users/cstone/Desktop/liquidhandling_Git_Clone/protocols/other/michael/created_files/round1_plate1_step2.hso)
		asp: F2 --> disp: B4
		asp: G1 --> disp: B5
		asp: H1 --> disp: B6
		asp: H12 --> disp: B7
	Dec